In [2]:
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn.functional as F
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch_geometric.transforms as T
from torch_geometric.utils import negative_sampling
from torch_geometric.data import DataLoader
from torch_geometric.nn import SAGEConv

Using backend: pytorch


In [3]:
dataset = PygNodePropPredDataset(name='ogbn-arxiv', 
                                 root='../../data/dataset/',
                                 transform=T.ToSparseTensor())
dataset

PygNodePropPredDataset()

In [4]:
torch.cuda.is_available()

True

In [5]:
data = dataset[0]
data

Data(adj_t=[169343, 169343, nnz=1166243], node_year=[169343, 1], x=[169343, 128], y=[169343, 1])

In [6]:
split_idx = dataset.get_idx_split()
split_idx

{'train': tensor([     0,      1,      2,  ..., 169145, 169148, 169251]),
 'valid': tensor([   349,    357,    366,  ..., 169185, 169261, 169296]),
 'test': tensor([   346,    398,    451,  ..., 169340, 169341, 169342])}

In [7]:
data.adj_t = data.adj_t.to_symmetric()
data.adj_t

SparseTensor(row=tensor([     0,      0,      0,  ..., 169341, 169342, 169342]),
             col=tensor([   411,    640,   1162,  ..., 163274,  27824, 158981]),
             size=(169343, 169343), nnz=2315598, density=0.01%)

In [8]:
row, col, _ = data.adj_t.coo()

In [9]:
edge_index = torch.stack([col, row], dim=0)
edge_index

tensor([[   411,    640,   1162,  ..., 163274,  27824, 158981],
        [     0,      0,      0,  ..., 169341, 169342, 169342]])

In [10]:
edge_index.t()

tensor([[   411,      0],
        [   640,      0],
        [  1162,      0],
        ...,
        [163274, 169341],
        [ 27824, 169342],
        [158981, 169342]])

In [11]:
edge_index.shape

torch.Size([2, 2315598])

In [12]:
edge_index[:, 0]

tensor([411,   0])

In [13]:
edge_index.t()[0]

tensor([411,   0])

In [124]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, out_channels,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x


class LinkPredictor(torch.nn.Module):
    def __init__(self, out_channels, bias=True):
        super(LinkPredictor, self).__init__()
        self.lin = torch.nn.Linear(1, out_channels, bias=bias)

    def reset_parameters(self):
        self.lin.weight.data.fill_(1)
        self.lin.bias.data.fill_(0)

    def forward(self, x_i, x_j):
        cos_sim = torch.sum(
            torch.mul(F.normalize(x_i), 
                      F.normalize(x_j)), 
            dim=1,
            keepdim=True
        )
        x = self.lin(cos_sim)
        return torch.sigmoid(x)

In [125]:
device=0
log_steps=1
num_layers=1
hidden_channels=50
dropout=0.5
batch_size=128 * 1024
lr=0.005
epochs=100
eval_steps=2
runs=10

In [126]:
device = f'cuda:{device}' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
device

device(type='cuda', index=0)

In [127]:
adj_t = data.adj_t.to(device)

In [128]:
torch.manual_seed(12345)

In [129]:
model = SAGE(hidden_channels, hidden_channels,
             dropout).to(device)

In [130]:
emb = torch.nn.Embedding(data.num_nodes, hidden_channels).to(device)

In [131]:
predictor = LinkPredictor(1).to(device)

In [132]:
predictor.lin.weight

Parameter containing:
tensor([[0.6750]], device='cuda:0', requires_grad=True)

In [133]:
emb.weight.size(0), emb.weight.size(1)

(169343, 50)

In [134]:
torch.nn.init.xavier_uniform_(emb.weight)

Parameter containing:
tensor([[ 3.8116e-03, -1.7777e-04,  2.6052e-03,  ..., -9.5538e-04,
         -5.6698e-03,  2.9117e-03],
        [ 4.8286e-03,  5.7555e-03,  4.1475e-03,  ...,  2.2556e-03,
          2.7162e-03,  4.5704e-03],
        [ 1.6921e-03, -1.5087e-03,  3.1802e-03,  ..., -4.9685e-03,
         -5.1306e-04, -9.3337e-04],
        ...,
        [ 4.1685e-03,  3.3565e-03,  3.5653e-03,  ...,  3.1168e-03,
         -1.2510e-03, -3.7040e-03],
        [-6.9713e-04,  2.8990e-03,  4.6939e-03,  ...,  3.2265e-03,
          2.1993e-03,  4.2817e-03],
        [ 4.2867e-03, -3.3813e-03,  3.8860e-05,  ...,  2.8638e-03,
          5.3680e-04, -6.6582e-05]], device='cuda:0', requires_grad=True)

In [135]:
model.reset_parameters()
predictor.reset_parameters()

In [136]:
predictor.lin.weight

Parameter containing:
tensor([[1.]], device='cuda:0', requires_grad=True)

In [137]:
predictor.lin.bias

Parameter containing:
tensor([0.], device='cuda:0', requires_grad=True)

In [138]:
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(emb.parameters()) +
    list(predictor.parameters()), lr=lr)

In [139]:
def train(model, predictor, x, adj_t, optimizer, batch_size):

    row, col, _ = adj_t.coo()
    edge_index = torch.stack([col, row], dim=0)

    model.train()
    predictor.train()

    pos_train_edge = edge_index.t().to(x.device)

    total_loss = total_examples = 0
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size,
                           shuffle=True):
        optimizer.zero_grad()

        h = model(x, adj_t)

        edge = pos_train_edge[perm].t()

        pos_out = predictor(h[edge[0]], h[edge[1]])
        pos_loss = -torch.log(pos_out + 1e-15).mean()

        edge = negative_sampling(edge_index, num_nodes=x.size(0),
                                 num_neg_samples=perm.size(0), method='sparse')

        neg_out = predictor(h[edge[0]], h[edge[1]])
        neg_loss = -torch.log(1 - neg_out + 1e-15).mean()

        loss = pos_loss + neg_loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(x, 1.0)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(predictor.parameters(), 1.0)

        optimizer.step()

        num_examples = pos_out.size(0)
        total_loss += loss.item() * num_examples
        total_examples += num_examples

    return total_loss / total_examples

In [140]:
for epoch in range(1, 1 + epochs):
    loss = train(model, predictor, emb.weight, adj_t,
                 optimizer, batch_size)
    
    if epoch % eval_steps == 0:
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}')

Epoch: 02, Loss: 1.0788
Epoch: 04, Loss: 1.0019
Epoch: 06, Loss: 0.9404
Epoch: 08, Loss: 0.8842
Epoch: 10, Loss: 0.8337
Epoch: 12, Loss: 0.7876
Epoch: 14, Loss: 0.7456
Epoch: 16, Loss: 0.7078
Epoch: 18, Loss: 0.6732
Epoch: 20, Loss: 0.6418
Epoch: 22, Loss: 0.6130
Epoch: 24, Loss: 0.5866
Epoch: 26, Loss: 0.5624
Epoch: 28, Loss: 0.5395
Epoch: 30, Loss: 0.5182
Epoch: 32, Loss: 0.4978
Epoch: 34, Loss: 0.4799
Epoch: 36, Loss: 0.4625
Epoch: 38, Loss: 0.4464
Epoch: 40, Loss: 0.4309
Epoch: 42, Loss: 0.4167
Epoch: 44, Loss: 0.4031
Epoch: 46, Loss: 0.3910
Epoch: 48, Loss: 0.3790
Epoch: 50, Loss: 0.3674
Epoch: 52, Loss: 0.3573
Epoch: 54, Loss: 0.3475
Epoch: 56, Loss: 0.3379
Epoch: 58, Loss: 0.3291
Epoch: 60, Loss: 0.3201
Epoch: 62, Loss: 0.3115
Epoch: 64, Loss: 0.3040
Epoch: 66, Loss: 0.2961
Epoch: 68, Loss: 0.2891
Epoch: 70, Loss: 0.2824
Epoch: 72, Loss: 0.2755
Epoch: 74, Loss: 0.2694
Epoch: 76, Loss: 0.2629
Epoch: 78, Loss: 0.2572
Epoch: 80, Loss: 0.2519
Epoch: 82, Loss: 0.2468
Epoch: 84, Loss:

In [141]:
predictor.lin.weight

Parameter containing:
tensor([[6.5461]], device='cuda:0', requires_grad=True)

In [142]:
predictor.lin.bias

Parameter containing:
tensor([-3.0290], device='cuda:0', requires_grad=True)

In [143]:
model.eval()

SAGE(
  (convs): ModuleList(
    (0): SAGEConv(50, 50)
  )
)

In [144]:
emb.weight

Parameter containing:
tensor([[ 0.3819,  0.1922, -0.1587,  ...,  0.0416, -0.0627, -0.1687],
        [-0.1920, -0.0258, -0.0561,  ...,  0.2288, -0.0568,  0.0695],
        [ 0.0694, -0.1284,  0.1548,  ..., -0.0669,  0.1322, -0.2019],
        ...,
        [ 0.0605,  0.1047, -0.0364,  ..., -0.1115, -0.1010,  0.0672],
        [ 0.2273, -0.0060, -0.1404,  ...,  0.0610, -0.1124, -0.1006],
        [-0.0603, -0.0364,  0.1967,  ...,  0.1632, -0.2398,  0.0146]],
       device='cuda:0', requires_grad=True)

In [145]:
h = model(emb.weight, adj_t)

In [146]:
h

tensor([[-0.2372, -0.4194,  0.2084,  ...,  0.0710, -0.3547,  0.1303],
        [-0.0282, -1.4530, -1.0147,  ...,  0.0901, -0.0268, -0.5035],
        [ 0.1827,  0.4492,  0.7295,  ...,  0.4557,  0.0898, -0.7916],
        ...,
        [ 0.0659, -0.3972, -0.3447,  ..., -0.1438,  0.3730, -0.3034],
        [-0.0729, -0.4610,  0.0285,  ...,  0.1237, -0.4734,  0.2572],
        [ 0.3317,  1.4260, -0.7288,  ..., -0.5351,  0.9650,  0.6444]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [147]:
predictor(h[0:2], h[411:413])

tensor([[0.8980],
        [0.0664]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [148]:
predictor(h[0:2], h[2:4])

tensor([[0.0938],
        [0.0200]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [149]:
(h[0] * h[2]).sum()

tensor(0.9522, device='cuda:0', grad_fn=<SumBackward0>)

In [150]:
(h[0] * h[411]).sum()

tensor(5.5484, device='cuda:0', grad_fn=<SumBackward0>)

In [151]:
np.save('../../models/graphsage_link_pred/full_graphsage_linkpred_43_h.npy', h.cpu().detach().numpy())
torch.save(model, '../../models/graphsage_link_pred/43_model.pt')
torch.save(emb, '../../models/graphsage_link_pred/43_embedding.pt')
torch.save(predictor, '../../models/graphsage_link_pred/43_predictor.pt')